In [16]:
import torch.nn as nn
import torch
import torch.optim
from torchvision.utils import save_image #保存图片
from torchvision.datasets import CIFAR10 #下载图片数据集
from torch.utils.data import DataLoader #读取批次
import torchvision.transforms as transforms #张量转换
from torch.autograd import Variable
import time #计时

In [2]:
#读取数据集
dataset = CIFAR10(root = 'D:/pythonwork/pytorch专栏/神经网络与pytorch实战/pytorch-book-master/chapter10_GAN/data', 
                 download = True, transform = transforms.ToTensor()) #下载数据集
dataloader = DataLoader(dataset, batch_size= 64, shuffle= True) #按批次读取数据(一批64张，总共有50000张，所以有50000/64=781批)，shuffle= True打乱数据

Files already downloaded and verified


In [3]:
#构建鉴别网络
n_d_feature = 64 #潜在大小
n_channel = 3 #输入通道数
dnet = nn.Sequential(
        nn.Conv2d(n_channel, n_d_feature, kernel_size=4,
                 stride=2, padding=1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(4 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(4 * n_d_feature, 1, kernel_size=4)).cuda()
print(dnet)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)


In [4]:
#构建生成网络
latent_size = 64 #输入通道数
n_g_feature = 64 #输出通道数

gnet = nn.Sequential(
        nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4,
                          bias=False),
        nn.BatchNorm2d(4 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=4,
                          stride=2, padding=1),
        nn.Sigmoid()).cuda()
print(gnet)

Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): Sigmoid()
)


In [5]:
#网络初始化
import torch.nn.init as init
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)
        
gnet.apply(weights_init)
dnet.apply(weights_init)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)

In [6]:
#载入cpu训练的预参数
checkpoint_d = torch.load('discriminator.pth', map_location=lambda storage, loc: storage.cuda(0))
checkpoint_g = torch.load('generator.pth', map_location=lambda storage, loc: storage.cuda(0))
dnet.load_state_dict(checkpoint_d)
gnet.load_state_dict(checkpoint_g)

FileNotFoundError: [Errno 2] No such file or directory: 'discriminator_1.pth'

In [12]:
#载入gpu训练的预参数
checkpoint_d = torch.load('D.pth')
checkpoint_g = torch.load('G.pth')
dnet.load_state_dict(checkpoint_d)
gnet.load_state_dict(checkpoint_g)

In [13]:
#定义损失
criterion = nn.BCEWithLogitsLoss().cuda()
#定义优化器
goptimizer = torch.optim.Adam(gnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))

In [14]:
#生成噪音数据，输入到G网络的数据
batch_size = 64
fixed_noise = torch.randn(batch_size, latent_size, 1, 1).cuda()
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [17]:
#开始训练
start = time.time() #开始时间

epoch_num = 5#共训练5个周期
for epoch in range(epoch_num):
    for batch_idx, data in enumerate(dataloader):
        real_images, _ = data
        batch_size = real_images.shape[0]
        #训练判别器D
        labels = torch.ones(batch_size)#真实数据的标签：1
        preds = dnet(Variable(real_images.type(Tensor))) #将真实数据喂给D网络
        outputs = preds.reshape(-1) #转换成未知行
        dloss_real = criterion(outputs, labels.type(Tensor))
        dmean_real = outputs.sigmoid().mean() #计算判别器将多少真数据判别为真，仅用于输出显示
        
        noises = torch.randn(batch_size, latent_size, 1, 1)
        fake_images = gnet(noises.type(Tensor)) #生成假数据
        labels = torch.zeros(batch_size)#生成假数据的标签：0
        fake = fake_images.detach() #类似于固定生成器参数
        preds = dnet(fake) #将假数据喂给判别器
        outputs = preds.reshape(-1)#转换成未知行
        dloss_fake = criterion(outputs.type(Tensor), labels.type(Tensor))
        dmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        dloss = dloss_real + dloss_fake #总的鉴别器损失为两者之和
        dnet.zero_grad()#梯度清零
        dloss.backward()#反向传播
        doptimizer.step()
        
        #训练生成器G
        labels = torch.ones(batch_size)#在训练生成器G时，希望生成器的标签为1
        preds = dnet(fake_images)#让假数据通过鉴别网络
        outputs = preds.reshape(-1)#转换成未知行
        gloss = criterion(outputs.type(Tensor), labels.type(Tensor))
        gmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        gnet.zero_grad()#梯度清零
        gloss.backward()#反向传播
        goptimizer.step()
        
        #输出本步训练结果
        print('[{}/{}]'.format(epoch, epoch_num) + '[{}/{}]'.format(batch_idx, len(dataloader)) +
             '鉴别器G损失:{:g} 生成器D损失：{:g}'.format(dloss, gloss) + 
             '真数据判真比例：{:g} 假数据判真比例：{:g}/{:g}'.format(dmean_real, dmean_fake, gmean_fake))
        if batch_idx % 100 == 0:
            fake = gnet(fixed_noise) #噪声生成假数据
            path = './data_new/gpu{:02d}_batch{:03d}.png'.format(epoch, batch_idx)
            save_image(fake, path, normalize=False)
            
end = time.time()
print((end - time_open)/60) #输出结束时间(单位：分钟)

[0/5][0/782]鉴别器G损失:0.992228 生成器D损失：1.14708真数据判真比例：0.559713 假数据判真比例：0.266662/0.349206
[0/5][1/782]鉴别器G损失:0.898733 生成器D损失：2.11418真数据判真比例：0.780303 假数据判真比例：0.438108/0.142552
[0/5][2/782]鉴别器G损失:0.704286 生成器D损失：1.67282真数据判真比例：0.653376 假数据判真比例：0.208841/0.216472
[0/5][3/782]鉴别器G损失:0.851468 生成器D损失：1.99456真数据判真比例：0.742065 假数据判真比例：0.388826/0.156253
[0/5][4/782]鉴别器G损失:0.970705 生成器D损失：0.986768真数据判真比例：0.520282 假数据判真比例：0.226215/0.400153
[0/5][5/782]鉴别器G损失:0.881173 生成器D损失：1.90667真数据判真比例：0.782878 假数据判真比例：0.433525/0.170355
[0/5][6/782]鉴别器G损失:0.757607 生成器D损失：1.95662真数据判真比例：0.709795 假数据判真比例：0.302739/0.164151
[0/5][7/782]鉴别器G损失:0.880779 生成器D损失：1.05235真数据判真比例：0.567993 假数据判真比例：0.223963/0.387138
[0/5][8/782]鉴别器G损失:1.00975 生成器D损失：2.29775真数据判真比例：0.799454 假数据判真比例：0.505663/0.121982
[0/5][9/782]鉴别器G损失:1.15434 生成器D损失：1.01413真数据判真比例：0.423139 假数据判真比例：0.188027/0.405523
[0/5][10/782]鉴别器G损失:0.7863 生成器D损失：2.17141真数据判真比例：0.847246 假数据判真比例：0.419298/0.14047
[0/5][11/782]鉴别器G损失:0.777875 生成器D损失：2.24311真数据判真比例：0.704341 假数据判真比例：

[0/5][96/782]鉴别器G损失:1.17218 生成器D损失：0.953276真数据判真比例：0.401804 假数据判真比例：0.143518/0.438714
[0/5][97/782]鉴别器G损失:1.0741 生成器D损失：2.65147真数据判真比例：0.873459 假数据判真比例：0.568507/0.0906171
[0/5][98/782]鉴别器G损失:0.859566 生成器D损失：1.51031真数据判真比例：0.541189 假数据判真比例：0.15894/0.253968
[0/5][99/782]鉴别器G损失:1.0325 生成器D损失：1.2027真数据判真比例：0.640152 假数据判真比例：0.393316/0.330519
[0/5][100/782]鉴别器G损失:0.894414 生成器D损失：2.088真数据判真比例：0.774747 假数据判真比例：0.427584/0.151797
[0/5][101/782]鉴别器G损失:0.88512 生成器D损失：1.4272真数据判真比例：0.593446 假数据判真比例：0.252085/0.271963
[0/5][102/782]鉴别器G损失:0.795079 生成器D损失：1.35337真数据判真比例：0.670768 假数据判真比例：0.286956/0.285567
[0/5][103/782]鉴别器G损失:0.981427 生成器D损失：2.15641真数据判真比例：0.787125 假数据判真比例：0.473718/0.133354
[0/5][104/782]鉴别器G损失:0.833398 生成器D损失：1.35719真数据判真比例：0.557928 假数据判真比例：0.171097/0.28561
[0/5][105/782]鉴别器G损失:0.753588 生成器D损失：1.98423真数据判真比例：0.822692 假数据判真比例：0.391402/0.153394
[0/5][106/782]鉴别器G损失:0.612121 生成器D损失：1.88584真数据判真比例：0.700366 假数据判真比例：0.199374/0.169981
[0/5][107/782]鉴别器G损失:0.867424 生成器D损失：1.1913真数据判真比例：0.6085

[0/5][191/782]鉴别器G损失:1.04397 生成器D损失：1.69656真数据判真比例：0.620129 假数据判真比例：0.36812/0.222156
[0/5][192/782]鉴别器G损失:0.804969 生成器D损失：1.22476真数据判真比例：0.632765 假数据判真比例：0.245762/0.323758
[0/5][193/782]鉴别器G损失:1.01428 生成器D损失：2.54848真数据判真比例：0.839471 假数据判真比例：0.508961/0.104406
[0/5][194/782]鉴别器G损失:0.995718 生成器D损失：1.1096真数据判真比例：0.490523 假数据判真比例：0.158428/0.378702
[0/5][195/782]鉴别器G损失:1.05419 生成器D损失：1.60388真数据判真比例：0.741531 假数据判真比例：0.469532/0.235549
[0/5][196/782]鉴别器G损失:0.677583 生成器D损失：2.16855真数据判真比例：0.74819 假数据判真比例：0.290545/0.132462
[0/5][197/782]鉴别器G损失:0.898017 生成器D损失：1.22686真数据判真比例：0.547717 假数据判真比例：0.207039/0.327384
[0/5][198/782]鉴别器G损失:0.883168 生成器D损失：1.71447真数据判真比例：0.75636 假数据判真比例：0.417767/0.208322
[0/5][199/782]鉴别器G损失:0.816737 生成器D损失：2.0392真数据判真比例：0.724238 假数据判真比例：0.350838/0.152833
[0/5][200/782]鉴别器G损失:0.788515 生成器D损失：1.37513真数据判真比例：0.577705 假数据判真比例：0.158941/0.278016
[0/5][201/782]鉴别器G损失:0.914239 生成器D损失：2.04567真数据判真比例：0.776484 假数据判真比例：0.446653/0.15046
[0/5][202/782]鉴别器G损失:0.653373 生成器D损失：2.05422真数据判真比例：

[0/5][286/782]鉴别器G损失:0.879784 生成器D损失：1.49436真数据判真比例：0.714201 假数据判真比例：0.385737/0.254816
[0/5][287/782]鉴别器G损失:0.892151 生成器D损失：2.26522真数据判真比例：0.749193 假数据判真比例：0.42187/0.122541
[0/5][288/782]鉴别器G损失:0.825232 生成器D损失：1.6748真数据判真比例：0.640973 假数据判真比例：0.25133/0.21383
[0/5][289/782]鉴别器G损失:1.16189 生成器D损失：0.874592真数据判真比例：0.502859 假数据判真比例：0.300642/0.452636
[0/5][290/782]鉴别器G损失:0.888846 生成器D损失：2.32762真数据判真比例：0.853033 假数据判真比例：0.469552/0.122033
[0/5][291/782]鉴别器G损失:0.764991 生成器D损失：1.97754真数据判真比例：0.6612 假数据判真比例：0.250706/0.174382
[0/5][292/782]鉴别器G损失:1.01865 生成器D损失：0.926076真数据判真比例：0.538931 假数据判真比例：0.248177/0.423779
[0/5][293/782]鉴别器G损失:0.979672 生成器D损失：2.28829真数据判真比例：0.815217 假数据判真比例：0.499558/0.118344
[0/5][294/782]鉴别器G损失:0.852559 生成器D损失：1.3156真数据判真比例：0.527636 假数据判真比例：0.132275/0.296954
[0/5][295/782]鉴别器G损失:0.807227 生成器D损失：1.97297真数据判真比例：0.829637 假数据判真比例：0.416423/0.164738
[0/5][296/782]鉴别器G损失:0.883522 生成器D损失：1.75038真数据判真比例：0.660238 假数据判真比例：0.317488/0.203582
[0/5][297/782]鉴别器G损失:0.64655 生成器D损失：1.62048真数据判真比例

[0/5][380/782]鉴别器G损失:1.03507 生成器D损失：0.88941真数据判真比例：0.460534 假数据判真比例：0.150252/0.437166
[0/5][381/782]鉴别器G损失:0.929664 生成器D损失：1.99385真数据判真比例：0.829989 假数据判真比例：0.480903/0.159322
[0/5][382/782]鉴别器G损失:0.805367 生成器D损失：2.29748真数据判真比例：0.706423 假数据判真比例：0.326781/0.122567
[0/5][383/782]鉴别器G损失:0.825335 生成器D损失：1.28817真数据判真比例：0.58623 假数据判真比例：0.190152/0.310674
[0/5][384/782]鉴别器G损失:0.78439 生成器D损失：1.68522真数据判真比例：0.759901 假数据判真比例：0.361617/0.223161
[0/5][385/782]鉴别器G损失:0.829296 生成器D损失：1.87592真数据判真比例：0.695579 假数据判真比例：0.327928/0.179912
[0/5][386/782]鉴别器G损失:0.706032 生成器D损失：1.83613真数据判真比例：0.71293 假数据判真比例：0.262846/0.196219
[0/5][387/782]鉴别器G损失:0.715093 生成器D损失：2.29155真数据判真比例：0.798974 假数据判真比例：0.35922/0.117619
[0/5][388/782]鉴别器G损失:0.735647 生成器D损失：1.38681真数据判真比例：0.609122 假数据判真比例：0.160895/0.287819
[0/5][389/782]鉴别器G损失:0.883989 生成器D损失：1.61881真数据判真比例：0.726281 假数据判真比例：0.396498/0.22599
[0/5][390/782]鉴别器G损失:0.925438 生成器D损失：1.60219真数据判真比例：0.632391 假数据判真比例：0.326888/0.227823
[0/5][391/782]鉴别器G损失:1.11708 生成器D损失：1.71126真数据判真比

...

[4/5][455/782]鉴别器G损失:0.720303 生成器D损失：2.10954真数据判真比例：0.828439 假数据判真比例：0.365567/0.146867
[4/5][456/782]鉴别器G损失:0.836766 生成器D损失：1.43651真数据判真比例：0.593456 假数据判真比例：0.211566/0.282973
[4/5][457/782]鉴别器G损失:0.797032 生成器D损失：2.57378真数据判真比例：0.85682 假数据判真比例：0.42559/0.0964031
[4/5][458/782]鉴别器G损失:0.725196 生成器D损失：2.00641真数据判真比例：0.657784 假数据判真比例：0.212145/0.16668
[4/5][459/782]鉴别器G损失:0.858 生成器D损失：1.09277真数据判真比例：0.575817 假数据判真比例：0.206901/0.379369
[4/5][460/782]鉴别器G损失:0.83628 生成器D损失：2.47961真数据判真比例：0.848904 假数据判真比例：0.445555/0.102846
[4/5][461/782]鉴别器G损失:0.846404 生成器D损失：1.94865真数据判真比例：0.636482 假数据判真比例：0.269321/0.168445
[4/5][462/782]鉴别器G损失:0.709112 生成器D损失：1.55541真数据判真比例：0.659356 假数据判真比例：0.180026/0.254226
[4/5][463/782]鉴别器G损失:0.659219 生成器D损失：2.19929真数据判真比例：0.824971 假数据判真比例：0.329466/0.144801
[4/5][464/782]鉴别器G损失:0.602968 生成器D损失：2.14371真数据判真比例：0.750572 假数据判真比例：0.237263/0.14215
[4/5][465/782]鉴别器G损失:0.802347 生成器D损失：1.6245真数据判真比例：0.663515 假数据判真比例：0.265844/0.232001
[4/5][466/782]鉴别器G损失:0.53199 生成器D损失：2.15229真数据判真比例：

[4/5][549/782]鉴别器G损失:0.860395 生成器D损失：1.35088真数据判真比例：0.666624 假数据判真比例：0.310826/0.281228
[4/5][550/782]鉴别器G损失:0.798911 生成器D损失：2.41319真数据判真比例：0.808459 假数据判真比例：0.40453/0.111442
[4/5][551/782]鉴别器G损失:0.640293 生成器D损失：1.99068真数据判真比例：0.691129 假数据判真比例：0.199671/0.157716
[4/5][552/782]鉴别器G损失:0.675611 生成器D损失：2.04733真数据判真比例：0.75588 假数据判真比例：0.283946/0.163772
[4/5][553/782]鉴别器G损失:0.691335 生成器D损失：1.36801真数据判真比例：0.654372 假数据判真比例：0.18355/0.291587
[4/5][554/782]鉴别器G损失:0.696743 生成器D损失：2.28931真数据判真比例：0.836473 假数据判真比例：0.351218/0.132483
[4/5][555/782]鉴别器G损失:0.640794 生成器D损失：2.35924真数据判真比例：0.755203 假数据判真比例：0.2599/0.115579
[4/5][556/782]鉴别器G损失:0.643678 生成器D损失：1.74823真数据判真比例：0.69254 假数据判真比例：0.200648/0.203625
[4/5][557/782]鉴别器G损失:0.881795 生成器D损失：1.38727真数据判真比例：0.636952 假数据判真比例：0.308926/0.298388
[4/5][558/782]鉴别器G损失:0.854659 生成器D损失：2.55069真数据判真比例：0.804947 假数据判真比例：0.420588/0.0999239
[4/5][559/782]鉴别器G损失:0.730371 生成器D损失：1.59818真数据判真比例：0.624497 假数据判真比例：0.180215/0.228472
[4/5][560/782]鉴别器G损失:1.0111 生成器D损失：0.935398真数据判真

[4/5][643/782]鉴别器G损失:0.762057 生成器D损失：2.30688真数据判真比例：0.692602 假数据判真比例：0.275691/0.118021
[4/5][644/782]鉴别器G损失:1.10393 生成器D损失：0.780091真数据判真比例：0.493532 假数据判真比例：0.229946/0.496826
[4/5][645/782]鉴别器G损失:1.0067 生成器D损失：3.02047真数据判真比例：0.866726 假数据判真比例：0.520957/0.0634329
[4/5][646/782]鉴别器G损失:0.864382 生成器D损失：1.31371真数据判真比例：0.528551 假数据判真比例：0.120272/0.311514
[4/5][647/782]鉴别器G损失:0.684216 生成器D损失：2.56515真数据判真比例：0.900976 假数据判真比例：0.401855/0.0928674
[4/5][648/782]鉴别器G损失:0.936146 生成器D损失：1.80235真数据判真比例：0.610339 假数据判真比例：0.264383/0.217745
[4/5][649/782]鉴别器G损失:0.778793 生成器D损失：1.21817真数据判真比例：0.63549 假数据判真比例：0.21234/0.35996
[4/5][650/782]鉴别器G损失:1.0425 生成器D损失：3.48597真数据判真比例：0.860138 假数据判真比例：0.539368/0.0407467
[4/5][651/782]鉴别器G损失:0.706633 生成器D损失：1.77084真数据判真比例：0.60209 假数据判真比例：0.0941896/0.208636
[4/5][652/782]鉴别器G损失:0.731994 生成器D损失：1.12172真数据判真比例：0.684893 假数据判真比例：0.250677/0.364153
[4/5][653/782]鉴别器G损失:0.798269 生成器D损失：2.99394真数据判真比例：0.835658 假数据判真比例：0.416856/0.0620753
[4/5][654/782]鉴别器G损失:0.445945 生成器D损失：2.38057真数

[4/5][737/782]鉴别器G损失:0.982143 生成器D损失：1.34846真数据判真比例：0.600046 假数据判真比例：0.301773/0.287922
[4/5][738/782]鉴别器G损失:0.705957 生成器D损失：2.80588真数据判真比例：0.827575 假数据判真比例：0.3579/0.0827044
[4/5][739/782]鉴别器G损失:0.915645 生成器D损失：0.96408真数据判真比例：0.509625 假数据判真比例：0.146995/0.415305
[4/5][740/782]鉴别器G损失:1.17334 生成器D损失：3.41293真数据判真比例：0.880082 假数据判真比例：0.575619/0.04842
[4/5][741/782]鉴别器G损失:0.93493 生成器D损失：1.60104真数据判真比例：0.492169 假数据判真比例：0.0889899/0.256748
[4/5][742/782]鉴别器G损失:0.594384 生成器D损失：1.36701真数据判真比例：0.75579 假数据判真比例：0.237773/0.29938
[4/5][743/782]鉴别器G损失:0.943975 生成器D损失：3.44629真数据判真比例：0.887741 假数据判真比例：0.500422/0.0434944
[4/5][744/782]鉴别器G损失:1.02812 生成器D损失：1.37775真数据判真比例：0.470108 假数据判真比例：0.112612/0.297197
[4/5][745/782]鉴别器G损失:0.567291 生成器D损失：1.67719真数据判真比例：0.797414 假数据判真比例：0.259931/0.231717
[4/5][746/782]鉴别器G损失:0.904435 生成器D损失：2.43426真数据判真比例：0.797424 假数据判真比例：0.436507/0.11102
[4/5][747/782]鉴别器G损失:0.793546 生成器D损失：1.65148真数据判真比例：0.608099 假数据判真比例：0.198817/0.215575
[4/5][748/782]鉴别器G损失:0.691471 生成器D损失：1.78068真数据判真

AttributeError: module 'time' has no attribute 'ti'

## 保存模型

In [20]:
torch.save(dnet.state_dict(),'./D.pth')
torch.save(gnet.state_dict(),'./G.pth')

17.5683580438296
